In [1]:
import os

In [5]:
%pwd

'c:\\Users\\Rohan Brahmakshatri\\Documents\\Python Scripts\\EndtoEnd_Project\\chicken_disease_classification_project'

In [4]:
os.chdir('..')

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [7]:
from cnnClassifiers.constants import *
from cnnClassifiers.utils.common import read_yaml,create_dictionaries

In [15]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_dictionaries([self.config.artifacts_root])
        
        
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config=self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_dictionaries([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [16]:
#update the components

import os
import urllib.request as request
import zipfile
from cnnClassifiers.utils.common import get_size
from cnnClassifiers import logger
import tensorflow as tf
import time


In [17]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [18]:
#update the pipeline

try:
    config=ConfigurationManager()
    prepare_callback_config=config.get_prepare_callback_config()
    prepare_callback=PrepareCallback(config=prepare_callback_config)
    callback_list=prepare_callback.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e


[2023-07-31 20:34:59,145: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-31 20:34:59,161: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-31 20:34:59,161: INFO: common: created directory at path :artifacts]


BoxKeyError: "'ConfigBox' object has no attribute 'prepare_callback'"